# Install Packages

In [1]:
#!pip install -U requests

In [2]:
#!pip install -U pyspark

In [3]:
#!pip3 install uszipcode

# Access dataset with API

In [4]:
import requests, json
import csv
import pandas as pd
from pyspark.sql import SparkSession
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [5]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext

#sc.stop()
sc = SparkContext() 
config = sc.getConf()
config.set('spark.cores.max','4')
config.set('spark.executor.memory', '8G')
config.set('spark.driver.maxResultSize', '8g')
config.set('spark.kryoserializer.buffer.max', '512m')
config.set("spark.driver.cores", "4")

sc.stop()

sc = SparkContext(conf = config) 
sqlContext = SQLContext(sc)

23/04/18 13:45:07 WARN Utils: Your hostname, ElsadeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.206.54.163 instead (on interface en0)
23/04/18 13:45:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/18 13:45:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
#Function for convert lat and lon to zipcode

from uszipcode import SearchEngine
import pandas as pd
import numpy as np

# define a function to get the zip code from a pair of coordinates
search = SearchEngine()
def get_zipcode(lat, long):
    result = search.by_coordinates(lat, long, radius=1, returns=1)
    if result:
        return result[0].zipcode
    else:
        return None

#use nympy to faster the process
get_zipcode_v = np.vectorize(lambda lat, long: get_zipcode(lat, long))

# Airbnb Data

## NYC

In [7]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("Pandas to PySpark").getOrCreate()

23/04/18 13:45:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
url = 'http://data.insideairbnb.com/united-states/ny/new-york-city/2023-03-06/visualisations/listings.csv'

nyc_air = requests.get(url)

filename = 'nyc_listings.csv'

with open(filename, 'wb') as f:
  f.write(nyc_air.content)

nycbnb = pd.read_csv(filename)
nycbnb.head()


# Convert the Pandas DataFrame to a PySpark DataFrame
rdd = spark.sparkContext.parallelize(nycbnb.to_dict('records'))
dfnyc = spark.createDataFrame(rdd)

dfnyc = dfnyc.select('name', 'latitude', 'longitude')

# Show the PySpark DataFrame
dfnyc.show()


23/04/18 13:45:39 WARN TaskSetManager: Stage 0 contains a task of very large size (1625 KiB). The maximum recommended task size is 1000 KiB.
23/04/18 13:45:48 WARN TaskSetManager: Stage 1 contains a task of very large size (1625 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------+---------+
|                name|latitude|longitude|
+--------------------+--------+---------+
|Skylit Midtown Ca...|40.75356|-73.98559|
|     BlissArtsSpace!|40.68535|-73.95512|
|Cozy Clean Guest ...| 40.8038|-73.96751|
|Large Furnished R...|40.76457|-73.98317|
|Large Sunny Brook...|40.66265|-73.99454|
|Comfortable, Sunn...|40.68292|-73.96381|
|Rooftop Deck/City...|40.76076|-73.96156|
|Lovely, Cozy, Roo...|40.66801|-73.98784|
|Most Central Loca...| 40.7672|-73.98464|
|Only 2 stops to M...|40.70935|-73.95342|
|Uptown Sanctuary ...|40.80107|-73.94255|
|Central Park 1BR ...|40.79544|-73.94836|
|Luminous Beautifu...|40.73405|-74.00281|
|Cozy Garden Apart...|40.71516|-73.96448|
|Sanctuary in East...| 40.6327|-73.93184|
|Room in the heart...|40.76336|-73.92099|
|UES Beautiful Blu...|40.78778|-73.94759|
|Room with En Suit...|40.68818|-73.96383|
|Huge Private  Flo...|40.68513|-73.96648|
|Amazing location!...|40.71248|-73.95881|
+--------------------+--------+---

## Austin

In [9]:
url = 'http://data.insideairbnb.com/united-states/tx/austin/2023-03-16/visualisations/listings.csv'
austin_air = requests.get(url)

filename = 'austin_listings.csv'

with open(filename, 'wb') as f:
  f.write(austin_air.content)

austinbnb = pd.read_csv(filename)
austinbnb.head()

# Convert the Pandas DataFrame to a PySpark DataFrame
rdd_aus = spark.sparkContext.parallelize(austinbnb.to_dict('records'))
dfaus = spark.createDataFrame(rdd_aus)

dfaus = dfaus.select('name', 'latitude', 'longitude')

# Show the PySpark DataFrame
dfaus.show()

+--------------------+--------+---------+
|                name|latitude|longitude|
+--------------------+--------+---------+
|Walk to 6th, Rain...|30.26057|-97.73441|
|      NW Austin Room|30.45697|-97.78422|
|Gem of a Studio n...|30.24885|-97.73587|
|Secluded Studio @...|30.26034|-97.76487|
|Woodland Studio L...|30.23466|-97.73682|
|Historic house in...|30.26098|-97.73072|
|3br/2ba Austin WF...|30.19756|-97.78754|
|Garage Apartment ...|30.23614|-97.73225|
|Clarksville Large...|30.28074|-97.75381|
|SXSW pad to crash...|30.25123|-97.70463|
|Lovely Central Au...|30.33771|-97.73708|
|OUTDOOR LIVING IN...|30.35123|-97.76207|
|Cozy Room in Nort...|30.37783|-97.70749|
|    Scooby Doo House|30.28588|-97.75145|
|Zilker's Peaceful...|30.25756|-97.76995|
|Colorful and Quir...|30.26146|-97.77276|
|Live Beneath Live...| 30.2309|-97.76619|
|South 1st Garage ...|30.23644|-97.76611|
|1 Bedroom Apt; Ab...|30.24648|-97.74913|
|Private, Detached...|  30.313|-97.75066|
+--------------------+--------+---

## Chicago

In [10]:
url = 'http://data.insideairbnb.com/united-states/il/chicago/2023-03-19/visualisations/listings.csv'

chic_air = requests.get(url)

filename = 'Chicago_listings.csv'

with open(filename, 'wb') as f:
  f.write(chic_air.content)

chicbnb = pd.read_csv(filename)
chicbnb.head()

rdd_chi = spark.sparkContext.parallelize(chicbnb.to_dict('records'))
dfchi = spark.createDataFrame(rdd_chi)

dfchi = dfchi.select('name', 'latitude', 'longitude')

# Show the PySpark DataFrame
dfchi.show()

+--------------------+--------+---------+
|                name|latitude|longitude|
+--------------------+--------+---------+
|Hyde Park - Walk ...| 41.7879| -87.5878|
|Comfy Garden Suit...| 41.9796|-87.66512|
|Trendy Roscoe Vil...|41.94342|-87.68121|
|Tiny Studio Apart...|41.90166|-87.68021|
|Best in Chicago, ...|41.92918|-87.70219|
|Historic Pullman ...|41.68843|-87.60712|
|Pullman Art Studi...|41.68954|-87.60786|
|3 Bedroom Across ...|41.94842|-87.65307|
|The Biddle House ...|41.91196|-87.63981|
|4 Bedroom Across ...|41.94774|-87.65421|
|Pullman School Ho...|41.68912|-87.60725|
|Lincoln Park Gues...|41.92357|-87.64947|
|Lovely Bedroom 3 ...|41.95758| -87.7271|
|Close to Michigan...|41.85923|-87.62198|
|Studio Apartment ...| 42.0076| -87.6825|
|4 Bedroom Across ...| 41.9475| -87.6542|
|LARGE Private 1BR...|41.77738|-87.60041|
| Musician's Quarters|41.79989|-87.59432|
|Steps From the Be...|41.88413|-87.64315|
|Elegant Lakefront...|41.93264|-87.64311|
+--------------------+--------+---

## Los Angeles

In [11]:
url = 'http://data.insideairbnb.com/united-states/ca/los-angeles/2023-03-07/visualisations/listings.csv'

LA_air = requests.get(url)

filename = 'LA_listings.csv'

with open(filename, 'wb') as f:
  f.write(LA_air.content)

LAbnb = pd.read_csv(filename)

rdd_la = spark.sparkContext.parallelize(LAbnb.to_dict('records'))
dfla = spark.createDataFrame(rdd_la)

dfla = dfla.select('name', 'latitude', 'longitude')

# Show the PySpark DataFrame
dfla.show()

23/04/18 13:47:23 WARN TaskSetManager: Stage 6 contains a task of very large size (1663 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------+------------+
|                name|  latitude|   longitude|
+--------------------+----------+------------+
|Phillips Ranch ro...|34.0318556|-117.7779937|
|Family oriented home|  34.43925|  -118.44409|
|        方便易居驿站|  34.04848|  -117.76887|
|TERRANEA OCEANFRN...|  33.73842|   -118.3958|
|Brand New Duplex ...|  33.74842|    -118.311|
| Zuma Malibu Retreat|  34.03979|  -118.86885|
|Single BR with Ki...|  33.73928|  -118.29911|
|Premium & New rem...|  33.99764|  -117.78979|
|Relaxing and priv...|  33.90055|  -118.26036|
|Unique Starry Nig...|  34.68979|  -117.77503|
|Awesome new and i...|  33.83587|  -118.28795|
|( VAL VERDE ) Spa...|  34.44707|  -118.67733|
|Million Dollar Vi...|  33.78223|  -118.31712|
|Amy’s FrontHouse ...|    33.984|  -118.05205|
|A Perfect Brand N...|  33.89942|  -118.04855|
|3 Bedrooms 4 beds...|  33.79169|  -118.34688|
|1. Private Rm, La...| 34.681137|  -118.08123|
|#7Artesia_ Disney...|  33.86314|   -118.0753|
|        Master Sui

23/04/18 13:47:23 WARN TaskSetManager: Stage 7 contains a task of very large size (1663 KiB). The maximum recommended task size is 1000 KiB.


## San Francisco

In [12]:
url = 'http://data.insideairbnb.com/united-states/ca/san-francisco/2023-03-06/visualisations/listings.csv'

sf_air = requests.get(url)

filename = 'sf_listings.csv'

with open(filename, 'wb') as f:
  f.write(sf_air.content)

sfbnb = pd.read_csv(filename)

rdd_sf = spark.sparkContext.parallelize(sfbnb.to_dict('records'))
dfsf = spark.createDataFrame(rdd_sf)

dfsf = dfsf.select('name', 'latitude', 'longitude')

# Show the PySpark DataFrame
dfsf.show()

+--------------------+--------+----------+
|                name|latitude| longitude|
+--------------------+--------+----------+
|Bright, Modern Ga...|37.77028|-122.43317|
|  Creative Sanctuary|37.74474|-122.42089|
|Friendly Room Apt...|37.76555|-122.45213|
|Historic Alamo Sq...|37.77564|-122.43642|
|Mission Sunshine,...| 37.7603|-122.42197|
|Elegant & Cozy w/...| 37.7175|-122.39698|
|Stylish, Spacious...| 37.7844|-122.47932|
|Mission's "La Len...|37.74766|-122.42041|
|Classic Nob Hill ...|37.79249|-122.41499|
|Sunny/Sunset view...|37.76901|-122.44701|
|Central San Franc...|37.77196|-122.43477|
|Sunny/Sunset view...|37.76932|-122.44723|
|Perfectly located...|37.76237|-122.42992|
|Lake Street Singl...|37.78546|-122.46148|
|Loft-like Apt./Ga...|37.74916|-122.42971|
|Prime Location - ...|37.79233|-122.42431|
|Best Views in all...|37.74633|-122.44591|
|Spacious 1brm w/ ...|37.72833|-122.44125|
|Lower Haight Urba...|37.77247|-122.43301|
|Comfortable space...|37.77988|-122.48488|
+----------

## Union the dataframe

In [13]:
airbnb = dfnyc.union(dfaus).union(dfchi).union(dfla).union(dfsf)
airbnb.count()

23/04/18 13:47:34 WARN TaskSetManager: Stage 10 contains a task of very large size (1625 KiB). The maximum recommended task size is 1000 KiB.


114433

## Get the zip code for Airbnb

In [14]:
airbnb_df = airbnb.toPandas()
airbnb_zip = get_zipcode_v(airbnb_df['latitude'], airbnb_df['longitude'])

airbnb_df['zipcode'] = airbnb_zip
airbnb_df = airbnb_df[['name', 'zipcode']]
len(airbnb_df)

23/04/18 13:48:52 WARN TaskSetManager: Stage 13 contains a task of very large size (1625 KiB). The maximum recommended task size is 1000 KiB.


114433

In [15]:
airbnb_df.to_csv('airbnb.csv', index=False)

# Crime data

## NYC

In [52]:
response_ny = requests.get('https://data.cityofnewyork.us/resource/uip8-fykc.csv?$limit=189774')

nycrime = 'ny_crime.csv'

with open(nycrime, 'wb') as f:
  f.write(response_ny.content)


nyc_crime_sdf = sqlContext.read.option("header", "true") \
                         .option("delimiter", ",") \
                         .option("inferSchema", "true") \
                         .csv(nycrime)

# NYC crime data clean

#time
from pyspark.sql.functions import col, date_format
nyc_crime_sdf = nyc_crime_sdf.withColumn("am_pm", date_format(col("arrest_date"), "a"))

#keep only the crime,"am_pm" longitude, and latitude
nyc_crime_sdf = nyc_crime_sdf.select("ofns_desc","am_pm", "longitude", "latitude").withColumnRenamed("ofns_desc", "crime")

#add city column
from pyspark.sql.functions import lit
nyc_crime_sdf = nyc_crime_sdf.withColumn("city", lit("New_York"))

#drop all the null
nyc_crime_sdf = nyc_crime_sdf.na.replace("(null)", None)
nyc_crime_sdf = nyc_crime_sdf.dropna()

## Austin

In [53]:
response_aus = requests.get('https://data.austintexas.gov/resource/fdj4-gpfu.csv?$limit=2371193&$where=occ_date_time%20%3E%3D%20%272022-01-01T00:00:00%27')

auscrime = 'aus_crime.csv'

with open(auscrime, 'wb') as f:
  f.write(response_aus.content)


auscrime_sdf = sqlContext.read.option("header", "true") \
                         .option("delimiter", ",") \
                         .option("inferSchema", "true") \
                         .csv(auscrime)

# austin crime data clean

#time
from pyspark.sql.functions import col, date_format
auscrime_sdf = auscrime_sdf.withColumn("am_pm", date_format(col("occ_date_time"), "a"))

#keep only the crime,"am_pm" longitude, and latitude
auscrime_sdf = auscrime_sdf.select("crime_type","am_pm", "zip_code").withColumnRenamed("crime_type", "crime").withColumnRenamed("zip_code", "zipcode")

#add city column
from pyspark.sql.functions import lit
auscrime_sdf = auscrime_sdf.withColumn("city", lit("Austin"))

#drop all the null
auscrime_sdf = auscrime_sdf.na.replace("(null)", None)
auscrime_sdf = auscrime_sdf.dropna()

## LA

In [54]:
response_la = requests.get('https://data.lacity.org/resource/2nrs-mtv8.csv?$limit=699280&$where=date_occ%20%3E%3D%20%272022-01-01T00:00:00%27')

lacrime = 'la_crime.csv'

with open(lacrime, 'wb') as f:
  f.write(response_la.content)


lacrime_sdf = sqlContext.read.option("header", "true") \
                         .option("delimiter", ",") \
                         .option("inferSchema", "true") \
                         .csv(lacrime)

# LA crime data clean

#time
from pyspark.sql.functions import col, date_format
lacrime_sdf = lacrime_sdf.withColumn("am_pm", date_format(col("date_occ"), "a"))

#keep only the crime,"am_pm" longitude, and latitude
lacrime_sdf = lacrime_sdf.select("crm_cd_desc","am_pm", "lon","lat").withColumnRenamed("crm_cd_desc", "crime").withColumnRenamed("lat", "latitude").withColumnRenamed("lon", "longitude")

#add city column
from pyspark.sql.functions import lit
lacrime_sdf = lacrime_sdf.withColumn("city", lit("LA"))

#drop all the null
lacrime_sdf = lacrime_sdf.na.replace("(null)", None)
lacrime_sdf = lacrime_sdf.dropna()

## Chicago

In [55]:
response_chi = requests.get('https://data.cityofchicago.org/resource/x2n5-8w5q.csv?$limit=247370&$where=date_of_occurrence%20%3E%3D%20%272022-01-01T00:00:00%27')

chicrime = 'chi_crime.csv'

with open(chicrime, 'wb') as f:
  f.write(response_chi.content)


chicrime_sdf = sqlContext.read.option("header", "true") \
                         .option("delimiter", ",") \
                         .option("inferSchema", "true") \
                         .csv(chicrime)

# Chicago crime data clean

#time
from pyspark.sql.functions import col, date_format
chicrime_sdf = chicrime_sdf.withColumn("am_pm", date_format(col("date_of_occurrence"), "a"))

#keep only the crime,"am_pm" longitude, and latitude
chicrime_sdf = chicrime_sdf.select("_primary_decsription","am_pm", "longitude","latitude").withColumnRenamed("_primary_decsription", "crime")

#add city column
from pyspark.sql.functions import lit
chicrime_sdf = chicrime_sdf.withColumn("city", lit("Chicage"))

#drop all the null
chicrime_sdf = chicrime_sdf.na.replace("(null)", None)
chicrime_sdf = chicrime_sdf.dropna()

## San Francisco

In [56]:
response_sf = requests.get('https://data.sfgov.org/resource/wg3w-h783.csv?$limit=717040&$where=incident_date%20%3E%3D%20%272022-01-01T00:00:00%27')

sfcrime = 'sf_crime.csv'

with open(sfcrime, 'wb') as f:
  f.write(response_sf.content)


sfcrime_sdf = sqlContext.read.option("header", "true") \
                         .option("delimiter", ",") \
                         .option("inferSchema", "true") \
                         .csv(sfcrime)

# San Fransico crime data clean

#time
from pyspark.sql.functions import col, date_format
sfcrime_sdf = sfcrime_sdf.withColumn("am_pm", date_format(col("incident_datetime"), "a"))

#keep only the crime,"am_pm" longitude, and latitude
sfcrime_sdf = sfcrime_sdf.select("incident_category","am_pm", "longitude","latitude").withColumnRenamed("incident_category", "crime")

#add city column
from pyspark.sql.functions import lit
sfcrime_sdf = sfcrime_sdf.withColumn("city", lit("SF"))

#drop all the null
sfcrime_sdf = sfcrime_sdf.na.replace("(null)", None)
sfcrime_sdf = sfcrime_sdf.dropna()

## Union all the crime data

In [57]:
crime_data_NOaus = nyc_crime_sdf.union(lacrime_sdf).union(chicrime_sdf).union(sfcrime_sdf)

## Get the zip code for Crime data

In [58]:
crime_data_NOaus_df = crime_data_NOaus.toPandas()
crime_data_NOaus_zip = get_zipcode_v(crime_data_NOaus_df['latitude'], crime_data_NOaus_df['longitude'])

crime_data_NOaus_df['zipcode'] = crime_data_NOaus_zip
crime_data_NOaus_df = crime_data_NOaus_df[['crime','am_pm','city', 'zipcode']]
crime_data_NOaus_df = crime_data_NOaus_df.dropna()


crime_data_aus_df = auscrime_sdf.toPandas()
crime_data = pd.concat([crime_data_NOaus_df, crime_data_aus_df], ignore_index=True)

In [59]:
crime_data['crime'] = crime_data['crime'].str.lower()

In [78]:
crime_data.tail()

AttributeError: 'list' object has no attribute 'tail'

In [61]:
crime_data.to_csv('crime_data.csv', index=False)

# Join Airbnb and Crime dataset

In [62]:
airbnb = pd.read_csv('airbnb.csv')
crime = pd.read_csv('crime_data.csv')

# MongoDB

In [29]:
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27017")
db = client.apan5400

In [ ]:
#collection.drop()

In [69]:
#collection1.drop()

In [67]:
with open('crime_data.csv', 'r', encoding='utf-8') as csvfile:
    # Create a reader object to read the CSV file
    reader = csv.DictReader(csvfile)

    # Create an empty list to hold the JSON objects
    crime_data = []

    # Iterate over each row in the CSV file
    for row in reader:
        # Add the row as a dictionary to the list
        crime_data.append(row)

In [70]:
collection1 = db.crime
collection1.insert_many(crime_data)

# Crime list function

In [33]:
def details(zipcode):
    crimelist = list(collection1.aggregate([
    {"$match": {"zipcode": zipcode}},
    {"$group": {"_id": "$crime","total_crime_events": { "$sum": 1 }}},
        { "$sort": { "total_crime_events": -1 } }])
                    )
    
    return pd.DataFrame(crimelist)

# Day/nigh crime table

In [34]:
import pymongo
import matplotlib.pyplot as plt
import tempfile

def DayNight_info(zipcode):
    results = list(collection1.aggregate([
        {'$match': {'zipcode': zipcode}},
        {'$group': {'_id': '$am_pm','count': {'$sum': 1}}}]))
    
    labels = [r['_id'] for r in results]
    counts = [r['count'] for r in results]

    if len(results) == 1 and labels[0] == "AM":
        return ("This city's open data has not yet updated day/night crime information")      

    return pd.DataFrame(results)

In [35]:
import pymongo

def DayNight_info(zipcode):
    results = list(collection1.aggregate([
        {'$match': {'zipcode': zipcode}},
        {'$group': {'_id': '$am_pm','count': {'$sum': 1}}}]))
    
    counts = [r['count'] for r in results]
    total = sum(counts)

    if len(results) == 1 and results[0]['_id'] == "AM":
        return "This city's open data has not yet updated day/night crime information"
      
    AM_pct = 100 * counts[0] / total
    PM_pct = 100 * counts[1] / total

    return f"Around this area, The crimes that happened in the morning are {AM_pct:.2f}%, and the crimes that happened at night are {PM_pct:.2f}%."


# Top 5 cirme

In [73]:
def get_top5_crime(zipcode):
    
    y = int(zipcode)
    x = crime.loc[crime['zipcode'] == y, 'city'].unique()
    city = x[0]
    top5 = list(collection1.aggregate([
    {"$match": {"city": city}},
    {"$group": {"_id": "$crime","total_crimes": { "$sum": 1 }}},
    { "$sort": { "total_crimes": -1 } },
    { "$limit": 5},
    { "$project": {"_id":0, "crime": "$_id","number": "$total_crimes"}}])
               )

    return pd.DataFrame(top5), city

In [79]:
crime.tail()

,crime,am_pm,city,zipcode
837542,family disturbance,PM,Austin,78751
837543,harassment,AM,Austin,78704
837544,theft by shoplifting,PM,Austin,78759
837545,doc offensive gesture,PM,Austin,78758
837546,burglary non residence,AM,Austin,78758


# Flask

In [98]:
from flask import Flask, request, render_template, Markup
app = Flask("Interactive App")

@app.route('/')
def my_form():
    return render_template("5400Proj.html")

@app.route('/', methods=['POST'])
def my_form_post():
    val = str(request.form['zipcode'])
    name = request.form['airbnb_name']

    if name == 'none':
        zipcode = val
    else:
        x = airbnb.loc[airbnb['name'] == name, 'zipcode']
        zipcode = x.iloc[0]

    
    result = details(zipcode)
    table_html = result.to_html(index=False)
    
    AM_PM_info = DayNight_info(zipcode)
    
    top5, city = get_top5_crime(zipcode)
    table_html2 = top5.to_html(index=False)
    
    return render_template('result.html', city = city, table = Markup(table_html), AMPMinfo = AM_PM_info, top5 = Markup(table_html2))

In [ ]:
app.run(host='localhost', port=5042)

 * Serving Flask app 'Interactive App'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5042
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 15:43:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 15:43:19] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 15:43:25] "POST / HTTP/1.1" 200 -
